# 02 – Entities & Slot Filler


Uses `Microsoft.Recognizers.Text.*` for dates and simple maps for roles/departments.


In [ ]:
//r "nuget: Microsoft.Recognizers.Text.DateTime, 1.8.13"

using Microsoft.Recognizers.Text;
using Microsoft.Recognizers.Text.DateTime;
using System.Text.RegularExpressions;
using semantic_kernel_app;
using Microsoft.Extensions.Logging;

public class SlotFiller : ISlotFiller
{
    private readonly ILogger<SlotFiller> _log;
    private static readonly string[] Depts = new[]{"engineering","marketing","sales","people","hr","it"};
    private static readonly (string keyword, string role)[] RoleMap = new[] {
        ("manager","Manager"),
        ("director","Director"),
        ("lead","Lead"),
        ("supervisor","Supervisor"),
        ("engineer","Engineer"),
    };

    public SlotFiller(ILogger<SlotFiller> log) => _log = log;

    public (Slots slots, double confidence, string? clarification) Fill(string query)
    {
        var q = query.ToLowerInvariant();

        // Names: simple split by non-letters, keep capitalized guesses too in a real system
        var names = Regex.Matches(q, @"[a-z]+").Select(m => m.Value)
            .Where(w => new[]{"rick","morty","summer","beth","jerry"}.Contains(w)).Distinct().ToArray();

        // Department
        string? dept = Depts.FirstOrDefault(d => q.Contains(d));

        // Role (first match)
        string? role = RoleMap.FirstOrDefault(r => q.Contains(r.keyword)).role;

        // Operator
        string? op = null;
        if (q.Contains("before")) op = "before";
        else if (q.Contains("after")) op = "after";
        else if (q.Contains("between")) op = "between";
        else if (q.Contains("since")) op = "since";

        // Date(s)
        DateTime? single = null;
        (DateTime, DateTime)? range = null;

        var results = DateTimeRecognizer.RecognizeDateTime(query, Culture.English);
        // Very lightweight interpretation for demo
        var years = Regex.Matches(q, @"\b(19|20)\d{2}\b").Select(m => int.Parse(m.Value)).ToList();
        if (years.Count >= 2)
        {
            var y1 = years.Min(); var y2 = years.Max();
            range = (new DateTime(y1,1,1), new DateTime(y2,12,31));
        }
        else if (years.Count == 1)
        {
            var y = years[0];
            single = new DateTime(y,1,1);
        }

        var slots = new Slots(
            Names: names.Length>0 ? names : null,
            Date: single,
            Range: range,
            Operator: op,
            Department: dept,
            Role: role
        );

        var conf = 0.5 + (names.Length>0?0.1:0) + (dept!=null?0.1:0) + (role!=null?0.1:0) + ((single!=null||range!=null)?0.2:0);
        string? clar = null;
        if (op=="between" && range==null) clar = "Between which dates? e.g., 2020 and 2022";
        return (slots, Math.Min(conf,0.95), clar);
    }
}

// demo
var sf = new SlotFiller(LoggerFactory.Create(b=>b.AddConsole()).CreateLogger<SlotFiller>());
var (slots, conf, clar) = sf.Fill("Show managers in engineering hired after 2020");
Console.WriteLine(System.Text.Json.JsonSerializer.Serialize(slots));
Console.WriteLine($"confidence={conf} clarify={clar}");
